In [1]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [14]:
# Veri artırma
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)


In [15]:
train_generator = datagen.flow_from_directory(
    'input/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)


Found 4747 images belonging to 4 classes.


In [16]:
val_generator = datagen.flow_from_directory(
    'input/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 1185 images belonging to 4 classes.


In [17]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [18]:
for layer in base_model.layers:
    layer.trainable = False


In [19]:
# Yeni sınıflandırıcı katmanlar
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(4, activation='softmax')(x)

In [20]:
model = Model(inputs=base_model.input, outputs=output)

In [21]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_generator, validation_data=val_generator, epochs=28, workers=4,  # veri işleyen CPU sayısı
                                                     use_multiprocessing=True)    # çoklu işlem kullan)

In [ ]:
loss, accuracy = model.evaluate(val_generator)
print("Validation Accuracy:", accuracy)
